<a href="https://colab.research.google.com/github/surabhipandey18/FoodAgri/blob/main/crop_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
df = pd.read_csv('/content/drive/MyDrive/Hackathon/Crop Prediction dataset.csv')
df.head()

,State_Name,District_Name,Crop_Year,Season,Crop,Temperature,Humidity,Soil_Moisture,Area,Production
0,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Arecanut,36,35,45,1254.0,2000.0
1,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Other Kharif pulses,37,40,46,2.0,1.0
2,Andaman and Nicobar Islands,NICOBARS,2000,Kharif,Rice,36,41,50,102.0,321.0
3,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Banana,37,42,55,176.0,641.0
4,Andaman and Nicobar Islands,NICOBARS,2000,Whole Year,Cashewnut,36,40,54,720.0,165.0


In [3]:
df.dropna(inplace=True)

In [4]:
df = df.drop('Crop_Year', axis = 1)
df.head()

,State_Name,District_Name,Season,Crop,Temperature,Humidity,Soil_Moisture,Area,Production
0,Andaman and Nicobar Islands,NICOBARS,Kharif,Arecanut,36,35,45,1254.0,2000.0
1,Andaman and Nicobar Islands,NICOBARS,Kharif,Other Kharif pulses,37,40,46,2.0,1.0
2,Andaman and Nicobar Islands,NICOBARS,Kharif,Rice,36,41,50,102.0,321.0
3,Andaman and Nicobar Islands,NICOBARS,Whole Year,Banana,37,42,55,176.0,641.0
4,Andaman and Nicobar Islands,NICOBARS,Whole Year,Cashewnut,36,40,54,720.0,165.0


In [31]:
categorical_col = ['State_Name', 'District_Name', 'Season', 'Crop']
numerical_col = ['Temperature', 'Humidity', 'Soil_Moisture', 'Area']
target = 'Production'
df.shape

(49784, 9)

In [6]:
X = df[categorical_col + numerical_col]
y = df[target]

In [7]:
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numerical_col),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_col)
])

In [30]:
X_processed = preprocessor.fit_transform(X)
X_processed.shape

(49784, 209)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

In [25]:
model = tf.keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [26]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['r2_score'])
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 128)            │        26,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 37,249 (145.50 KB)

 Trainable params: 37,249 (145.50 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
996/996 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 123358087741440.0000 - r2_score: -0.0015 - val_loss: 205532119957504.0000 - val_r2_score: -0.0012
Epoch 2/100
996/996 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 184399370911744.0000 - r2_score: -0.0012 - val_loss: 205444710662144.0000 - val_r2_score: -7.7200e-04
Epoch 3/100
996/996 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 171062625042432.0000 - r2_score: -0.0018 - val_loss: 205215366119424.0000 - val_r2_score: 3.4517e-04
Epoch 4/100
996/996 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 165078494085120.0000 - r2_score: -0.0160 - val_loss: 204842073063424.0000 - val_r2_score: 0.0022
Epoch 5/100
996/996 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 179535337226240.0000 - r2_score: -0.0280 - val_loss: 204232623915008.0000 - val_r2_score: 0.0051
Epoch 6/100
996/996 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 66104487575552.0000 - r2_score: -0.0447 - val_loss: 203356836462592.0000 - val_r2_score: 0.0094
Epoch 7/100
996/996 ━━━━━━━━━━━━━━━━━━━━ 3

In [28]:
loss, r2_score = model.evaluate(X_test, y_test)
print(f"Test MAE: {r2_score}")

312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 22532381999104.0000 - r2_score: 0.6936
Test MAE: 0.6465739011764526


In [29]:
model.save("yield_prediction.keras")